In [3]:
import pandas as pd
import numpy as np
import os # <<< [추가] 파일 및 폴더 관리를 위해 os 모듈 임포트

# 1. 처리할 공장 CSV 파일 목록과 샘플링 비율 설정
factory_files = [
    '../DB/icn.csv', 
    '../DB/hws.csv', 
    '../DB/kum.csv', 
    '../DB/ygs.csv'
]
# sampling_ratio = 1 로 설정하셨으므로, 사실상 모든 EPC를 추출하여 합치는 것과 같습니다.
# 이는 전체 데이터를 통합하는 올바른 방법입니다.
sampling_ratio = 0.2
random_seed = 42

# 2. 각 공장 파일에서 샘플링할 EPC 코드를 저장할 리스트
sampled_epc_codes = []

print("--- 1단계: 각 공장별 EPC 코드 샘플링 시작 ---")

for file in factory_files:
    try:
        print(f"'{file}' 파일 처리 중...")
        df_epc_only = pd.read_csv(file, usecols=['epc_code'], dtype=str)
        unique_epcs = df_epc_only['epc_code'].unique()
        
        n_samples = int(len(unique_epcs) * sampling_ratio)
        if n_samples == 0 and len(unique_epcs) > 0:
            n_samples = 1
        
        print(f"  - 고유 EPC 수: {len(unique_epcs)}")
        print(f"  - 샘플링할 EPC 수 ({sampling_ratio*100}%): {n_samples}")
        
        np.random.seed(random_seed)
        sampled_list = np.random.choice(unique_epcs, size=n_samples, replace=False).tolist()
        sampled_epc_codes.extend(sampled_list)
        
    except FileNotFoundError:
        print(f"경고: '{file}' 파일을 찾을 수 없습니다. 건너뜁니다.")
    except Exception as e:
        print(f"오류: '{file}' 처리 중 문제 발생 - {e}")

final_sampled_epcs = list(set(sampled_epc_codes))
print("\n--- 1단계 완료 ---")
print(f"모든 공장에서 샘플링된 총 고유 EPC 코드 수: {len(final_sampled_epcs)}")


# =================================================================
# 3. [최적화] 샘플링된 EPC에 해당하는 모든 이력(row)을 메모리 효율적으로 추출
# =================================================================
print("\n--- 2단계: 샘플링된 EPC에 해당하는 전체 이력 추출 시작 (메모리 최적화 적용) ---")

all_dataframes = []
chunk_size = 100000  # 파일을 10만 줄씩 나누어 읽음

for file in factory_files:
    try:
        print(f"'{file}' 파일에서 데이터 분할 로딩 중...")
        # 파일을 chunk_size 만큼씩 나누어 읽는 반복자(iterator) 생성
        for chunk in pd.read_csv(file, dtype=str, chunksize=chunk_size):
            # 각 chunk(작은 DataFrame)에서 필요한 EPC에 해당하는 행만 필터링
            filtered_chunk = chunk[chunk['epc_code'].isin(final_sampled_epcs)]
            # 필터링된 결과만 리스트에 추가
            if not filtered_chunk.empty:
                all_dataframes.append(filtered_chunk)
        
    except FileNotFoundError:
        print(f"경고: '{file}' 파일을 다시 로드하는 중 찾을 수 없습니다.")

# 모든 필터링된 DataFrame 조각들을 하나로 합칩니다.
if all_dataframes:
    final_sample_df = pd.concat(all_dataframes, ignore_index=True)
else:
    final_sample_df = pd.DataFrame()

print("\n--- 2단계 완료 ---")
print(f"최종 추출된 데이터 행(row) 수: {len(final_sample_df)}")

# =================================================================
# 4. 결과를 'dataSet' 폴더 안에 저장
# =================================================================
if not final_sample_df.empty:
    output_folder = 'dataSet'
    output_filename = 'factory_20pct_data.csv' # sampling_ratio=1이므로 모든 데이터를 의미하는 이름으로 변경
    
    # 폴더가 없으면 생성
    os.makedirs(output_folder, exist_ok=True)
    
    # 최종 파일 경로 생성 (폴더 + 파일명)
    full_path = os.path.join(output_folder, output_filename)
    
    final_sample_df.to_csv(full_path, index=False)
    print(f"\n✅ 완료: 모든 데이터가 '{full_path}' 파일로 저장되었습니다.")
else:
    print("\n⚠️ 경고: 추출된 데이터가 없어 파일을 생성하지 않았습니다.")

--- 1단계: 각 공장별 EPC 코드 샘플링 시작 ---
'../DB/icn.csv' 파일 처리 중...
  - 고유 EPC 수: 100000
  - 샘플링할 EPC 수 (20.0%): 20000
'../DB/hws.csv' 파일 처리 중...
  - 고유 EPC 수: 50000
  - 샘플링할 EPC 수 (20.0%): 10000
'../DB/kum.csv' 파일 처리 중...
  - 고유 EPC 수: 20000
  - 샘플링할 EPC 수 (20.0%): 4000
'../DB/ygs.csv' 파일 처리 중...
  - 고유 EPC 수: 30000
  - 샘플링할 EPC 수 (20.0%): 6000

--- 1단계 완료 ---
모든 공장에서 샘플링된 총 고유 EPC 코드 수: 40000

--- 2단계: 샘플링된 EPC에 해당하는 전체 이력 추출 시작 (메모리 최적화 적용) ---
'../DB/icn.csv' 파일에서 데이터 분할 로딩 중...
'../DB/hws.csv' 파일에서 데이터 분할 로딩 중...
'../DB/kum.csv' 파일에서 데이터 분할 로딩 중...
'../DB/ygs.csv' 파일에서 데이터 분할 로딩 중...

--- 2단계 완료 ---
최종 추출된 데이터 행(row) 수: 184602

✅ 완료: 모든 데이터가 'dataSet\factory_20pct_data.csv' 파일로 저장되었습니다.
